In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import linregress
from scipy.integrate import solve_ivp


Caudal_objetivo = [40, 50, 60]



Au = 0.1
bu = 0.1    
c0 = 0.0
V = 1.45  # Volumen del reactor en litros
uno_partido_T = []
valor_k = []
ln_k = []
medidas = [
    {'temperatura': [25.6, 25.6, 25.6, 25.6,25.6,25.6],
     'tiempos' : [0, 3, 6, 9, 12, 15,18],
     'FA': [40],
     'FB': [40],
     'T0': 25.6,
     'conductividad': [6.63/1000, 6.56/1000, 5.87/1000, 5.85/1000, 6.93/1000, 6.93/1000,6.98/1000]},

    {'temperatura': [25.6, 25.6, 25.6, 25.6, 25.6,25.6],
     'tiempos' : [23, 25, 27, 29, 31,33],
     'FA': [50],
     'FB': [50],
     'T0' : 25.6,
     'conductividad': [5.38/1000, 5.56/1000, 7.33/1000, 6.39/1000,  5.8/1000,5.8/1000]},

    {'temperatura': [34.2, 35, 34.8, 34.9, 34.9],
     'T0' : np.mean([35.1, 34.4, 34.4, 34.4, 34.4]),
     'FA': [60],
     'FB': [60],
     'tiempos' : [47, 50, 53, 56, 59],
     'conductividad': [7.48/1000, 7.47/1000, 7.45/1000, 7.45/1000, 7.46/1000]},
]

dataframes_medidas = {}
for i, medida in enumerate(medidas):
    df = pd.DataFrame(medida)
    # Asignar un nombre específico al DataFrame basado en la temperatura objetivo
    nombre_df = f"Temperatura_{Caudal_objetivo[i]}"
    dataframes_medidas[nombre_df] = df
    display(dataframes_medidas[nombre_df])

plt.figure(figsize=(8, 6))
colores = ['blue', 'orange', 'green']

# Usar enumerate para acceder al índice y las medidas
for i, medida in enumerate(medidas):
    Fa = medida['FA'][0]
    Fb = medida['FB'][0]
    a0 = (Fa / (Fa + Fb)) * Au
    b0 = (Fb / (Fa + Fb)) * bu
    T0 = medida['T0']+273.15

    if b0 < a0:
        cinf = b0 
    else:
        cinf = a0

    alpha_c_inf = 0.07 * (1 + 0.0284 * (T0 - 294)) * cinf
    alpha_a0 = 0.195 * (1 + 0.0184 * (T0 - 294)) * a0

    if c0 == 0:
        Alpha0 = alpha_a0 

    ainf = a0 - b0 if a0 >= b0 else 0
    alpha_a_inf = 0.195 * (1 + 0.0184 * (T0 - 294)) * ainf if ainf != 0 else 0
    alpha_inf = alpha_c_inf + alpha_a_inf

    
    C = medida['conductividad']
    T = medida['tiempos']

    at = []
    ct = []
    xa = []
    xe = []

#! REAJUSTAR APARTIR DE AQUI
# FIXME: 

    for c, temp, t in zip(C, Temp, T):
        at_val = (ainf - a0) * ((Alpha0 - c) / (Alpha0 - alpha_inf)) + a0
        ct_val = cinf * ((Alpha0 - c) / (Alpha0 - alpha_inf))
        xa_val = (a0 - at_val) / a0
        xe_val = ct_val / cinf
        
        # Almacenar los resultados
        at.append(at_val)
        ct.append(ct_val)
        xa.append(xa_val)
        xe.append(xe_val)
    
    # Crear un DataFrame con los resultados del bucle
    df_result = pd.DataFrame({
        'Tiempo, min': T,
        'conductividad': C,
        'Temperatura': Temp,
        'at': at,
        'ct': ct,
        'xa': xa,
        'xe': xe
    })
    # Calcular la media de las columnas
    media_at = df_result['at'].mean()
    media_ct = df_result['ct'].mean()
    media_xa = df_result['xa'].mean()
    media_xe = df_result['xe'].mean()
    media_Tiempo = df_result['Tiempo, min'].mean()
    media_conductividad = df_result['conductividad'].mean()

    # Agregar fila de medias al final del DataFrame
    df_media = pd.DataFrame({
        'Tiempo, min': ['Media'],
        'conductividad': [media_conductividad],  # Ningún valor para la columna de conductividad
        'Temperatura': [media_Tiempo],  # Ningún valor para la columna de Temperatura
        'at': [media_at],
        'ct': [media_ct],
        'xa': [media_xa],
        'xe': [media_xe]
    })

    # Concatenar la fila de medias al DataFrame original
    df_result = pd.concat([df_result, df_media], ignore_index=True)
    display(df_result)
    valor_media_at = df_result['at'].iloc[-1]
    k = ((Fa / 60000 + (Fb / 60000)) / V) * ((a0 - valor_media_at) / (valor_media_at ** 2))  # m^-1 s^-1
    valor_k = np.append(valor_k, k)  # Almacenar en valor_k
    uno_partido_T.append(1/(Temperatura_objetivo[i] + 273.15))
    ln_k.append(np.log(k))
    plt.plot(T, C, marker='o', linestyle='None', color=colores[i], label=f"Temperatura {Temperatura}")  # Cambia el label
    plt.plot(T, C, linestyle='--', color=colores[i])
    

# Configurar el título y etiquetas de la gráfica
plt.title('Conductividad frente a Tiempo')
plt.xlabel('Tiempo (min)')
plt.ylabel('Conductividad (S/m)')
plt.legend()  # Añadir la leyenda para identificar cada serie
plt.grid(True)  # Añadir una cuadrícula para mayor claridad
plt.show()  # Mostrar la gráfica

# Gráfica de ln(k) vs 1/T
plt.figure(figsize=(8, 6))
plt.plot(uno_partido_T, ln_k, marker='o', linestyle='None', color=colores[i], label='ln(k) vs 1/T')
plt.xlabel('1/T (K)')
plt.ylabel('ln(k)')
plt.legend()
plt.grid(True)
plt.show()
df_k = pd.DataFrame({
    'k (m^-1 s^-1)': valor_k,
    '1/T (K)': uno_partido_T,
    'ln(k)': ln_k
})
display(df_k)
slope, intercept, r_value, _, std_err = linregress(uno_partido_T, ln_k)

R = 8.314
Ea = -slope * R
A = np.exp(intercept)

df_arrhenius = pd.DataFrame({
    'Ea J/mol': [Ea],
    'A L/mol s': [A]
})
display(df_arrhenius)
Fa  = Fa/60000 #L/s
t_eval = np.linspace(0, 60, 500)
def reactor_edo(t, at, Fa, k, V, a0):
    da_dt = (Fa / V) * (a0 - at) - k * at
    return da_dt
for j in range(len(valor_k)):
    temperatura = Temperatura_objetivo[j]
    t_span = [0, 60]
    k_j = valor_k[j]  # Usar el valor correspondiente de k
    sol = solve_ivp(reactor_edo, t_span, [a0], args=(Fa, k_j, V, a0), t_eval=t_eval)
    t = sol.t
    a_t = sol.y[0]
    plt.plot(t, a_t, label=f'[NaOH] para T={temperatura}°C (mol/L)')  

plt.xlabel('Tiempo (min)')
plt.ylabel('Concentración de NaOH (mol/L)')
plt.title('Evolución de la concentración de NaOH en el reactor')
plt.legend()
plt.grid(True)
plt.show()
